Playground to test out different models for validation.

In [2]:
# Imports
from multi_modal_edge_ai.adl_inference.validate import split_and_validate
from multi_modal_edge_ai.adl_inference.parser import parse_file
from multi_modal_edge_ai.adl_inference.svm_model import SVMModel
import pandas as pd

data_A, ground_truth_A = parse_file("./public_dataset/OrdonezA_Sensors.csv",
                                    "./public_dataset/OrdonezA_ADLs.csv")
data_B, ground_truth_B = parse_file("./public_dataset/OrdonezB_Sensors.csv",
                                    "./public_dataset/OrdonezB_ADLs.csv")

In [7]:
model = SVMModel() # create svm model

avg, cm = split_and_validate(data_A,
                             ground_truth_A,
                             model,
                             window_length_seconds=300,
                             window_slide_seconds=150)

cm = pd.DataFrame(cm, columns=['Sleeping', 'Meal_Preparation', 'Kitchen_Usage', 'Bathroom_Usage', 'Idle', 'Relax', 'Outside'])
pd.set_option("display.max_columns", 10)
print(cm)
print(avg)


   Sleeping  Meal_Preparation  Kitchen_Usage  Bathroom_Usage  Idle  Relax   
0      1280                 0              0               0     0      0  \
1         0                37              0               1     5      0   
2         0                 0              0               0     0      0   
3         0                 1              0               8     3      0   
4         6                 6              0               7    34     35   
5         0                 1              0               0     2   1338   
6         0                 0              0               0     8      0   

   Outside  
0        0  
1       18  
2        0  
3       35  
4       79  
5        4  
6      272  
0.9336477987421383


In [17]:
from multi_modal_edge_ai.adl_inference.validate import validate
from multi_modal_edge_ai.adl_inference.window_splitter import split_into_windows

window_length_seconds = 300
window_slide_seconds = 150
windows = split_into_windows(data_A, ground_truth_A, window_length_seconds, window_slide_seconds)
split_index = int(len(windows) * 0.6)
train = windows[:split_index]
test = windows[split_index:]

# Define the hyperparameter grid
param_grid = {
    'n_components': [4, 6, 8],
    'max_iter': [1000, 2000, -1],
    'kernel': ['linear', 'rbf'],
}

best_score = 0
best_params = None

# Iterate over each combination of hyperparameters
for n_components in param_grid['n_components']:
    for max_iter in param_grid['max_iter']:
        for kernel in param_grid['kernel']:
            # Create an instance of the SVMModel with current hyperparameters
            model = SVMModel(n_components=n_components, max_iter=max_iter, kernel=kernel)
            score = validate(train, test, model)
            print(n_components, max_iter, kernel, score[0])
            # Check if the current score is better than the previous best score
            if score[0] > best_score:
                best_score = score[0]
                best_params = {
                    'n_components': n_components,
                    'max_iter': max_iter,
                    'kernel': kernel,
                }

# Print the best hyperparameters found
print("Best hyperparameters:", best_params)

# Print the best score achieved
print("Best score:", best_score)

/home/roach/PycharmProjects/multi-modal-edge-ai/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


4 1000 linear 0.9292452830188679
4 1000 rbf 0.9276729559748428


/home/roach/PycharmProjects/multi-modal-edge-ai/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


4 2000 linear 0.9292452830188679
4 2000 rbf 0.9276729559748428
4 -1 linear 0.9289308176100629
4 -1 rbf 0.9276729559748428


/home/roach/PycharmProjects/multi-modal-edge-ai/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


6 1000 linear 0.929559748427673
6 1000 rbf 0.9289308176100629


/home/roach/PycharmProjects/multi-modal-edge-ai/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


6 2000 linear 0.929559748427673
6 2000 rbf 0.9289308176100629
6 -1 linear 0.929559748427673
6 -1 rbf 0.9289308176100629


/home/roach/PycharmProjects/multi-modal-edge-ai/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


8 1000 linear 0.9323899371069182
8 1000 rbf 0.9314465408805032
8 2000 linear 0.9327044025157233
8 2000 rbf 0.9314465408805032
8 -1 linear 0.9327044025157233
8 -1 rbf 0.9314465408805032
Best hyperparameters: {'n_components': 8, 'max_iter': 2000, 'kernel': 'linear'}
Best score: 0.9327044025157233
